In [1]:
clean_up = True
%run stdPackages.ipynb

The file _gams_py_gdb0.gdx is still active and was not deleted.
The file _gams_py_gdb1.gdx is still active and was not deleted.


# Data arrangement, model A

Model A is a small open economy with two domestic production sectors, Ramsey-like consumers with CRRA utility and endogenous labor supply, and Armington-like trade.

In [2]:
name = 'A' # name of model
error = 1e-10 # tolerance when testing IO balance
IOv = f"{d['data']}\\A_IOv.xlsx" # file with value balance
IOp = f"{d['data']}\\A_IOp.xlsx" # file with prices
dur = f"{d['data']}\\A_dur.xlsx" # file with durable specifciation 
glob = CGE_globals.SmallOpen(kwargs_vals = {'t': range(2016,2051)}) # global settings used throughout; interest rates, long run growth rates, time index etc.

### 1: Load IO data on values

*Given the settings above, the following goes through some standard steps in setting up balanced IO data. Go to step 4 to add new exogenous data to the standard data*

In [3]:
ms = ['p','f','i','h','g']
db = GpyDB(**{'name': f"IO_{name}", 'alias': pd.MultiIndex.from_tuples([('n','nn'),('n','nnn'), ('s','ss')])})

*Load data and create some useful subsets:*
* ```f_x``` identifies sectors in module ```x```,
* ```nOut_x``` identifies outputs from module ```x```,
* ```nEqui``` identifies the goods for which we require en equilibrium to be established (see equilibrium module).
* ```d_qS``` identifies the sector, goods combinations that supplies ```nEqui[n]``` goods,
* ```d_qD``` identifies the sector, goods combinations that demands ```nEqui[n]``` goods.
* ```d_qSEqui``` identifies the sector, goods combinations that are exogenous in partial equilibrium, endogenous in general equilibrium.
* ```d_pEqui``` identifies the goods that are exogenous in partial equilibrium, endogenous in general equilibrium.

In [4]:
wv = read.simpleLoad(IOv)
dbs_v = {m: read.SeriesDB_from_wb(wv, {'variables': m}) for m in ms}
for m in ms:
    if 'vS' in dbs_v[m].database:
        dbs_v[m][f"s_{m}"] = dbs_v[m]['vS'].index.levels[0]
        dbs_v[m][f"nOut_{m}"] = dbs_v[m]['vS'].index.levels[-1]
dbs_v['f']['nOut_f'] = dbs_v['f'].get('vD').index.levels[-1].difference(dbs_v['p'].get('nOut_p'))
dbs_v['f']['s_f'] = dbs_v['f'].get('vD').index.levels[0].difference(dbs_v['p'].get('s_p'))

*Merge:*

In [5]:
[robust.robust_merge_dbs(db, db_i, priority='second') for db_i in dbs_v.values()];
robust.robust_merge_dbs(db, read.SeriesDB_from_wb(wv, {'maps': ['maps']})) # add mappings
db['nEqui'] = db['vS'].index.levels[-1]
db['d_qS']  = db['vS'].index
db['d_qD'] = adj.rc_pd(db['vD'], db['nEqui']).index
db['d_qSEqui'] = adj.rc_pd(db['d_qS'].vals, ('not', db['nOut_h']))
db['d_pEqui']  = db['nOut_h']

*Check value balances for domestic production sectors + investment sectors:*

In [6]:
vS = adj.rc_pd(db.get('vS'), ('or', [db.get('s_p'), db.get('s_i')])).groupby('s').sum()
vD_p = adj.rc_pd(db.get('vD'), ('and', [db.get('s_p'), ('not', db.get('nOut_i'))])).groupby('s').sum()
vD_i = adj.rc_pd(db.get('vD'), db.get('s_i')).groupby('s').sum()
vDiff = vS-vD_p.combine_first(vD_i)-adj.rc_pd(db.get('TotalTax'), ('or', [db.get('s_p'), db.get('s_i')]))
assert abs(vDiff).max()<error, f"The IO value data is not balanced check. The difference (vS-vD-TotalTax): {vDiff[abs(vDiff)>error]}"

### 2: IO data on prices

If no price data is added, add default price of 1 on all goods:

In [7]:
if IOp is None:
    db['p'] = pd.Series(1, index = db.get('vS').index.levels[-1].union(db.get('nOut_f')))
else:
    db['p'] = read.SeriesDB_from_wb(IOp, {'variables': ['pEqui']})['p']

Note: We do not automatically load any other prices than the equilibrium ones (i.e. regulation is not automatically loaded here).

### 3: Load data on durables

Load durables data: If a sheet name ends with 'V' the sheet loads variables, if 'M' it loads mappings.

In [8]:
wdur = read.simpleLoad(dur)
db_dur = read.SeriesDB_from_wb(wdur, {'variables': [sheet.title for sheet in wdur._sheets if sheet.title.endswith('V')],
                                      'maps': [sheet.title for sheet in wdur._sheets if sheet.title.endswith('M')]})

Read the subset of durables and investment goods from the mapping ```dur2inv```: 

In [9]:
db['dur_p'] = db_dur.get('dur2inv').droplevel('nn').unique() # what variables are durables (K)
db['inv_p'] = db_dur.get('dur2inv').droplevel('n').unique().rename({'nn':'n'}) # what variables are investment goods (I)
robust.robust_merge_dbs(db, db_dur, priority='second') # add to main database

Back out quantities:

In [10]:
db['qD'] = db.get('qD').combine_first( (adj.rc_pd(db.get('vD'), ('not', ('or', [db.get('svngs'), db.get('dur_p')]))) / db.get('p')) )
db['qS'] = db['vS'].vals / db.get('p')

Broadcast all variables using the time index (repeat variables for all $t$):

In [11]:
def addT(symbol, t):
    return adjMultiIndex.bc(symbol, t).reorder_levels(['t']+symbol.index.names if 't' not in symbol.index.names else symbol.index.names)
[db.__setitem__(k, addT(db.get(k), glob.db['t'].vals)) for k in db.getTypes(['variable','scalar_variable'])];

Initialize the prices ```pD``` and ```pS``` at the equilibrium prices:

In [12]:
db['pD'] = adjMultiIndex.bc(db.get('p'), db.get('qD')).reorder_levels(db['qD'].domains).rename('pD') # span the pure prices 'p' to fit entire qD domain
db['pS'] = adjMultiIndex.bc(adj.rc_pd(db.get('p'), ('not', db.get('nOut_f'))), db.get('qS')).reorder_levels(db['qS'].domains).rename('qS') # span 'p' to fit domain of qS. Drop prices on foreign goods.

### 4: Add other data

Load interest rates:

In [13]:
# robust.robust_merge_dbs(db, read.SeriesDB_from_wb(IOp, {'variables': ['interestRates']}), 'second')

## Export:

In [14]:
aggregateDB.readSets(db) # read sets from the symbols in data

In [15]:
db.export(repo = d['data'])
with open(f"{d['data']}\\glob_{name}", "wb") as file:
    pickle.dump(glob,file)